# Clustering DBSCAN

In [1]:
### 기본 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

'''
[step 1] 데이터 준비/기본 설정
'''

# 서울 시내 중학교 진학률 데이터셋
file_path = '2016_middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path, header=0)

# IPython Console 디스플레이 옵션 설정하기
pd.set_option('display.width', None)        # 출력화면의 너비
pd.set_option('display.max_rows', 100)      # 출력할 행의 개수 한도
pd.set_option('display.max_columns', 10)    # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth', 20)   # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True)   # 유니코드 사용 너비 조정

print(df.columns.values)

['Unnamed: 0' '지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고'
 '외고_국제고' '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']


In [2]:
'''
[Step 2] EDA 데이터 탐색
'''

# 데이터 살펴보기
print(df.head(3))
print('\n')

# 데이터 자료형 확인
print(df.info())
print('\n')

# 데이터 통계 요약 정보 확인
print(df.describe())

# 필요없는 데이터 삭제
df = df.drop(['Unnamed: 0'], axis=1)

   Unnamed: 0    지역                               학교명  코드  유형  ...  \
0           0  성북구  서울대학교사범대학부설중학교.....       3  국립  ...   
1           1  종로구  서울대학교사범대학부설여자중학교...     3  국립  ...   
2           2  강남구           개원중학교                     3  공립  ...   

  기타진학  취업   미상       위도        경도  
0    0.004     0  0.000  37.594942  127.038909  
1    0.031     0  0.000  37.577473  127.003857  
2    0.009     0  0.003  37.491637  127.071744  

[3 rows x 21 columns]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  415 non-null    int64  
 1   지역          415 non-null    object 
 2   학교명         415 non-null    object 
 3   코드          415 non-null    int64  
 4   유형          415 non-null    object 
 5   주야          415 non-null    object 
 6   남학생수        415 non-null    int64  
 7   여학생수        415 non-null    int64  
 8   일반고         4

In [3]:
# 지도에 위치 표시
# 서울 지도
mschool_map = folium.Map(location = [37.55, 126.98],
                        tiles = 'Stamen Terrain',
                        zoom_start=12)

# 중학교 위치 정보를 circle marker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                       radius = 5,        # 원의 반지름
                       color='brown',     # 원의 둘레 색상
                       fill=True,         
                       fill_color='coral',  # 원을 채우는 색
                       fill_opacity=0.7,    # 투명도
                       popup=name).add_to(mschool_map)
    
# 지도를 html 파일로 저장
mschool_map.save('mschool.html')

In [5]:
'''
[Step 3] 데이터 전처리
'''
# 원핫인코딩 (더미 변수)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 객체화
le = LabelEncoder()
oh = OneHotEncoder()

# type 함수 예약어여서 _ >> 변수 이름 통일 _ 
_code = le.fit_transform(df.코드)
_loc = le.fit_transform(df.지역)
_type = le.fit_transform(df.유형)
_day = le.fit_transform(df.주야)

# df 열 만들기
df['location'] = _loc
df['code'] = _code
df['type'] = _type
df['day'] = _day

df.head()

,지역,학교명,코드,유형,주야,...,경도,location,code,type,day
0,성북구,서울대학교사범대학부설중학교...,3,국립,주간,...,127.038909,16,0,1,0
1,종로구,서울대학교사범대학부설여자중학교...,3,국립,주간,...,127.003857,22,0,1,0
2,강남구,개원중학교,3,공립,주간,...,127.071744,0,0,0,0
3,강남구,개포중학교,3,공립,주간,...,127.062201,0,0,0,0
4,서초구,경원중학교,3,공립,주간,...,127.008900,14,0,0,0


In [7]:
'''
[Step 4] DBSCAN 군집 모형 - sklearn 사용
'''
# DBSCAN
from sklearn.cluster import *

# 분석에 사용할 속성 선택(과학고, 외고_국제고, 자사고)
columns_list_1 = [9, 10, 13]
x = df.iloc[:, columns_list_1]

# 정규화
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x = sc.fit_transform(x)

# 모델링 (DBSCAN) 
model = DBSCAN(eps=0.2, min_samples=5)

# 모형 학습
model.fit(x)

# 예측(군집 - 비지도)
cluster_label = model.labels_
print(cluster_label)

# 예측 결과를 데이터프레임에 추가
df['cluster'] = cluster_label
df.head(3)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  2 -1  0 -1
 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  0  3 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  1  0
 -1 -1 -1  0 -1 -1 -1 -1  0 -1  0  0 -1 -1  0 -1 -1 -1  0  0 -1 -1  0 -1
 -1 -1  0 -1 -1 -1  0  2  0  0  0  0  0 -1 -1 -1  0 -1  0 -1 -1  0 -1  0
 -1  0  0 -1 -1 -1 -1  1  0 -1  0  0 -1 -1 -1  0 -1 -1 -1 -1 -1  0  1 -1
 -1  0  2  0 -1 -1  1 -1 -1 -1  0  0  0 -1 -1  0 -1 -1 -1  0  0 -1 -1 -1
 -1  0 -1 -1 -1  0 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1  0 -1  0  0 -1 -1
 -1 -1 -1  0 -1 -1 -1  1  0  3  1 -1  0  0 -1  0 -1 -1  0  0  2 -1 -1  3
  0  0 -1 -1 -1 -1  0 -1  0  0 -1  0  0  0 -1 -1  0 -1 -1 -1 -1 -1  2  0
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0  0 -1 -1  0 -1  3  0  2 -1 -1
 -1 -1  0 -1 -1 -1  0 -1  0  0 -1 -1 -1 -1 -1  1 -1  0  1 -1  0  0  1 -1
  2 -1  0 -1 -1 -1 -1  0 -1 -1  1  0 -1  0 -1 -1  0  3  0 -1 -1 -1  2 -1
 -1 -1 -1  0  0  0  1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1

,지역,학교명,코드,유형,주야,...,location,code,type,day,cluster
0,성북구,서울대학교사범대학부설중학교...,3,국립,주간,...,16,0,1,0,-1
1,종로구,서울대학교사범대학부설여자중학교...,3,국립,주간,...,22,0,1,0,-1
2,강남구,개원중학교,3,공립,주간,...,0,0,0,0,-1


In [8]:
# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped_cols = [0, 1, 3] + columns_list_1   
'''
[0, 1, 3, 9, 10, 13]
지역 학교명 유형 과학고 외고_국제고 자사고
'''
grouped = df.groupby('cluster')
for key, group in grouped:    # key가 인덱스
    print('*key: ',key)         # df['cluster'] = cluster_label = model.labels_
    print('*number of group: ', len(group))
    print(group.iloc[:, grouped_cols].head())
    print('\n')

*key:  -1
*number of group:  255
     지역                               학교명  유형  과학고  외고_국제고  \
0  성북구  서울대학교사범대학부설중학교.....    국립   0.018        0.007   
1  종로구  서울대학교사범대학부설여자중학교...  국립   0.000        0.035   
2  강남구           개원중학교                  공립   0.009        0.012   
3  강남구           개포중학교                  공립   0.013        0.013   
4  서초구           경원중학교                  공립   0.007        0.010   

   자사고  
0   0.227  
1   0.043  
2   0.090  
3   0.065  
4   0.282  


*key:  0
*number of group:  102
      지역          학교명  유형  과학고  외고_국제고  자사고
13  서초구  동덕여자중학교  사립     0.0        0.022   0.038
22  강남구      수서중학교  공립     0.0        0.019   0.044
28  서초구      언남중학교  공립     0.0        0.015   0.050
34  강남구      은성중학교  사립     0.0        0.016   0.065
43  송파구      거원중학교  공립     0.0        0.021   0.054


*key:  1
*number of group:  45
         지역          학교명  유형  과학고  외고_국제고  자사고
46     강동구      동신중학교  사립     0.0          0.0   0.044
103    양천구      신원중학교  공립     0.0          0.0   

In [13]:
# 서울 지도
cluster_map = \
folium.Map(location=[37.55, 126.98],
          tiles = 'Stamen Terrain',
          zoom_start=12)

# 그래프로 표현 - 시각화
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan'}

for name, lat, lng, cluster in zip(df.학교명, df.위도, df.경도, df.cluster):
    folium.CircleMarker([lat, lng],
                       radius = 5,                    # 원의 반지름
                       color = colors[cluster],       # 원의 둘레 색상
                       fill = True,     
                       fill_color = colors[cluster],  # 원을 채우는 색
                       fill_opacity = 0.7,            # 투명도
                       popup = name).add_to(cluster_map)
    
# 지도를 html 파일로 저장하기
cluster_map.save('seoul_mschool_cluster.html')

In [9]:
# (과학고 외고국제고, 자사고 진학률 + 유형)

columns_list_2 = [9, 10, 13, 22]
x2 = df.iloc[:, columns_list_2]

# 정규화
x2 = sc.fit_transform(x2)

# 모델링 (DBSCAN)
dbm2 = DBSCAN(eps=0.2, min_samples=5)

# 학습
dbm2.fit(x2)

# 예측. 비지도학습이니까 라벨링
df['cluster2'] = dbm2.labels_

In [10]:
# 클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped2_cols = [0,1,3] + columns_list_2
'''
[0, 1, 3, 9, 10, 13, 22]
지역 학교명 유형 과학고 외고_국제고 자사고 + 유형
'''
grouped2 = df.groupby('cluster2')

for key, group in grouped2:
    print('*key: ', key)
    print('*number: ', len(group))
    print(group.iloc[:,grouped2_cols])
    print('\n')

*key:  -1
*number:  281
         지역                               학교명  유형  과학고  외고_국제고  \
0      성북구  서울대학교사범대학부설중학교.....    국립   0.018        0.007   
1      종로구  서울대학교사범대학부설여자중학교...  국립   0.000        0.035   
2      강남구           개원중학교                  공립   0.009        0.012   
3      강남구           개포중학교                  공립   0.013        0.013   
4      서초구           경원중학교                  공립   0.007        0.010   
..        ...                  ...                   ...     ...          ...   
376    마포구           창천중학교                  공립   0.007        0.020   
378  서대문구         가재울중학교                  공립   0.000        0.021   
379    성북구           길음중학교                  공립   0.008        0.033   
380    강서구           마곡중학교                  공립   0.000        0.024   
382    강남구           세곡중학교                  공립   0.004        0.021   

     자사고  type  
0     0.227     1  
1     0.043     1  
2     0.090     0  
3     0.065     0  
4     0.282     0  
..      ...   ...  
376

In [15]:
# 서울지도
cluster2_map = folium.Map(location=[37.55, 126.98],
                          tiles = 'Stamen Terrain',
                          zoom_start = 12)

# 그래프로 표현 - 시각화
for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.cluster2):
    folium.CircleMarker([lat, lng],
                       radius = 5,
                       color = colors[clus],
                       fill = True,
                       fill_color = colors[clus],
                       fill_opacity = 0.7,
                       popup=name).add_to(cluster2_map)
    
# 지도를 html 파일로 저장
cluster2_map.save('seoul_mschool_cluster2.html')

In [16]:
# X3 데이터셋에 대하여 위의 과정을 반복(과학고, 외고_국제고)
columns_list3 = [9, 10]
x3 = df.iloc[:, columns_list3]
print(x3[:5])
print('\n')

x3 = sc.fit_transform(x3)
dbm3 = DBSCAN(eps=0.2, min_samples=5)
dbm3.fit(x3)
dbm3.labels_
df['cluster3'] = dbm3.labels_

grouped3_cols = [0, 1, 3] + columns_list3
grouped3 = df.groupby('cluster3')
for key, group in grouped3:
    print('* key :', key)
    print('* number :', len(group))    
    print(group.iloc[:, grouped3_cols].head())
    print('\n')

cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)    

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.cluster3):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster3_map)
    
cluster3_map.save('seoul_mschool_cluster3.html')

   과학고  외고_국제고
0   0.018        0.007
1   0.000        0.035
2   0.009        0.012
3   0.013        0.013
4   0.007        0.010


* key : -1
* number : 61
     지역                             학교명  유형  과학고  외고_국제고
0  성북구  서울대학교사범대학부설중학교.....  국립   0.018        0.007
3  강남구           개포중학교                공립   0.013        0.013
6  강남구         압구정중학교                공립   0.015        0.036
7  강남구  단국대학교사범대학부속중학교.....  사립   0.032        0.005
8  강남구           대명중학교                공립   0.013        0.029


* key : 0
* number : 160
      지역                               학교명  유형  과학고  외고_국제고
1   종로구  서울대학교사범대학부설여자중학교...  국립     0.0        0.035
13  서초구       동덕여자중학교                  사립     0.0        0.022
22  강남구           수서중학교                  공립     0.0        0.019
28  서초구           언남중학교                  공립     0.0        0.015
29  강남구           언북중학교                  공립     0.0        0.007


* key : 1
* number : 111
      지역      학교명  유형  과학고  외고_국제고
2   강남구  개원중학교  공립   0.009        

In [17]:
cluster_map

In [18]:
cluster2_map

In [19]:
cluster3_map

---------------------

In [ ]:
le.fit_transform(df.코드)

# 3,5,9 >> 0,1,2  (뭔진 모르지만 일단 0,1,2 로 바꿔줌 (라벨링))
# 아마 0 >> 일반고?